<a href="https://colab.research.google.com/github/rebertmatheus/colab-notebooks/blob/master/Lab_Redes_Neurais_com_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB - Transfer learning

Neste exemplo, carregamos a rede MobileNetV2 pré treinada com o dataset da ImageNet, após carregar e testar o modelo, realizamos o transfer learning utilizando o dataset "cats VS Dogs" do tensorflow.

#Imports e inicializações

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
import numpy as np
from typing import List
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

# Carrega o modelo com pesos do ImageNet e sem a camada de classificação


In [2]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
check_model = MobileNetV2(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

Cria uma classe para usar os modelos e fazer um teste com algumas imagens da internet.

#Realizando o Transfer Learning com a rede Cats vs Dogs do TensorFlow

In [3]:
#Carrega o dataset do cats_vs_dogs

(train_ds, val_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:]'],
    as_supervised=True,
    with_info=True,
)

In [4]:
#Função para pré-processar o dataset

def preprocess(image, label):
  image = tf.image.resize(image, (224, 224))  # Redimensiona para o tamanho da InceptionV3
  image = preprocess_input(image)  # Aplica o pré-processamento da InceptionV3
  return image, label

In [5]:
train_ds = train_ds.map(preprocess).batch(32)
val_ds = val_ds.map(preprocess).batch(32)

In [6]:
#Criando o modelo

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
#Congelando as camadas iniciais

for layer in base_model.layers[:249]: #Congela as primeiras 249 camadas
  layer.trainable = False

In [8]:
#Compila e treina o modelo

model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(train_ds, epochs=10, validation_data=val_ds) # Epochs setado como 1 para agilizar o tempo de treinamento e minimizar a espera, o ideal seria deixar ao menos 10.

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1065s 2s/step - accuracy: 0.9752 - loss: 0.0751 - val_accuracy: 0.9901 - val_loss: 0.0342
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1048s 2s/step - accuracy: 0.9883 - loss: 0.0350 - val_accuracy: 0.9893 - val_loss: 0.0340
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1096s 2s/step - accuracy: 0.9914 - loss: 0.0253 - val_accuracy: 0.9897 - val_loss: 0.0358
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1100s 2s/step - accuracy: 0.9920 - loss: 0.0221 - val_accuracy: 0.9890 - val_loss: 0.0400
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1035s 2s/step - accuracy: 0.9932 - loss: 0.0197 - val_accuracy: 0.9897 - val_loss: 0.0428
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1044s 2s/step - accuracy: 0.9940 - loss: 0.0178 - val_accuracy: 0.9912 - val_loss: 0.0339
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1035s 2s/step - accuracy: 0.9957 - loss: 0.0126 - val_accuracy: 0.9908 - val_loss: 0.0377
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1112s 2s/step - accuracy: 0.9965 - loss: 0.0108 - 

In [12]:
# Plotando perda
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

# Plotando acurácia
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Accuracy over epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()


TypeError: 'History' object is not subscriptable

In [14]:
# Diretório para salvar o modelo
model_dir = 'saved_models'
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, 'cats_vs_dogs_transfer_learning.keras')
model.save(model_path)